# ДЗ 15. Синонимайзинг текста
Александров Валентин, 620 группа

## Подход

Для работы используются рускоязычные модели из пакета [Natasha](https://github.com/natasha/natasha). В качестве синонимов берем токен с наиболее похожим векторным представлением. Синонимы не подбираются для токенов, которые являются частью именнованных сущностей, которые нашла модель NER.
Также подобранный синоним не применяется, если он и исходное слово имеют различную морфологию или дистанция между ними превышает некоторое пороговое значение, подобранное вручную

In [1]:
texts = ["В Париже проходит массовая манифестация в поддержку свободы СМИ, передает корреспондент РИА Новости. Акция, организованная при поддержке крупнейших профсоюзов Франции, собрала огромное число участников. Площадь Республики, стартовая точка маршрута, полностью заполнена людьми. Колонна растянулась по бульвару Тампль и начала движение к площади Бастилии.",
"Военные эксперты, опрошенные РИА Новости, разошлись в оценке актуальности для России стратегических железнодорожных ракетных комплексов, аналогичных \"Молодцу\", принятому на вооружение ровно 31 год назад.",
"Союз европейских футбольных ассоциаций (УЕФА) определится с местом проведения финала Лиги наций 2021 года 3 декабря, сообщается на сайте организации. УЕФА 3 декабря проведет заседание исполкома, в повестке которого также будут рассмотрены вопросы по назначению мест проведения Лиги конференций — 2022 (новый турнир УЕФА) и финального турнира молодежного чемпионата Европы (среди игроков до 21 года) 2023 года."]

In [2]:
import gensim
import numpy as np
import pymystem3
from pathlib import Path
import zipfile
from gensim.models.fasttext import FastText
from string import punctuation

In [114]:
from slovnet import NER
from navec import Navec
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize, sentenize
import gc
from sklearn.metrics import pairwise_distances
from slovnet import Morph

Для работы с рускоязычным текстом будем использовать модули проекта [Natasha](https://github.com/natasha/natasha). Возьмем оттуда модель NER, модель для морфологии и модель для эмбеддингов, основанную на GloVe.

In [93]:
ner_model = NER.load('C:/Datasets/slovnet_ner_news_v1.tar')
navec_news = Navec.load('C:/Datasets/navec_news_v1_1B_250K_300d_100q.tar')
navec = Navec.load('C:/Datasets/navec_hudlit_v1_12B_500K_300d_100q.tar')
morph = Morph.load('C:/Datasets/slovnet_morph_news_v1.tar')
morph.navec(navec_news)
_ = ner_model.navec(navec_news)

Пример работы разметки NER модели:

In [81]:
markup = ner_model(texts[2])
show_markup(markup.text, markup.spans)

Союз европейских футбольных ассоциаций (УЕФА) определится с местом 
ORG──────────────────────────────────────────                      
проведения финала Лиги наций 2021 года 3 декабря, сообщается на сайте 
организации. УЕФА 3 декабря проведет заседание исполкома, в повестке 
             ORG─                                                    
которого также будут рассмотрены вопросы по назначению мест проведения
 Лиги конференций — 2022 (новый турнир УЕФА) и финального турнира 
                                       ORG─                       
молодежного чемпионата Европы (среди игроков до 21 года) 2023 года.


Подготовим таблицу с эмбеддингами для всех слов:

In [82]:
indexes = navec.pq.indexes[np.arange(navec.pq.vectors)]
parts = navec.pq.codes[navec.pq.qdims, indexes]
emb_matrix = parts.reshape(-1, 300)

del indexes, parts
gc.collect()

52252

Пример морфологического анализа:

In [176]:
text_tokenized = [token.text for token in tokenize(text)]
morph_markup = next(morph.map([text_tokenized]))
for token in markup.tokens[:9]:
    print(f'{token.text:>20} {token.tag}')

                   В ADP
              Париже PROPN|Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing
            проходит VERB|Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act
            массовая ADJ|Case=Nom|Degree=Pos|Gender=Fem|Number=Sing
        манифестация NOUN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
                   в ADP
           поддержку NOUN|Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
             свободы NOUN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
                 СМИ NOUN|Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur


In [177]:

def get_text_synonyms(text, verbose=True, metric='cosine', distance_threshold = 0.4):

    markup = ner_model(text)
    ne_starts = np.array([span.start for span in markup.spans])
    ne_stops  = np.array([span.stop for span in markup.spans])

    tokens_to_replace = {}

    text_tokenized = [token.text for token in tokenize(text)]
    morph_markup = next(morph.map([text_tokenized]))
    morph_tags = morph_markup.tags

    if verbose:
        print(f"{'Original token':20}       {'Closest token':20} {'value':6}   {'is_distance_ok?':^10}    {'is_morph_ok?':^10}")
        print("-"*85)

    for token, morph_tag in zip(list(tokenize(text)), morph_tags):

        # check if token is a part of named entity
        if np.any(np.logical_and(token.start >= ne_starts, token.stop <= ne_stops)):
            continue

        emb = navec.get(token.text)
        if emb is None:
            continue

        index = navec.vocab[token.text]
        distances = pairwise_distances(emb_matrix, emb_matrix[index:index+1], metric=metric)

        sorted_indecies = distances[:,0].argsort()
        sorted_values = distances[sorted_indecies,0]

        synonym_index = sorted_indecies[1]
        synonym = navec.vocab.words[synonym_index]
        
        is_distance_ok = sorted_values[1] < distance_threshold
        

        synonym_morph = next(morph.map([[synonym]]))
        synonym_morph_tag = synonym_morph.tags[0]

        is_morph_ok = synonym_morph_tag == morph_tag

        is_ok = is_morph_ok and is_distance_ok
        if is_ok:
            tokens_to_replace[token] = synonym

        if verbose:
            print(f"{token.text:20}       {synonym:20} {sorted_values[1]:.6f}   {'x' if is_distance_ok else '-':^10}   {'x' if is_morph_ok else '-':^10}")

    return tokens_to_replace

def paste_synonyms(text, tokens_to_replace):

    new_text = text

    bias = 0

    for token, synonym in tokens_to_replace.items():

        token_len = token.stop - token.start
        new_text = new_text[:token.start+bias] + synonym + new_text[token.stop+bias:]

        bias += len(synonym) - token_len

    return new_text

## Эксперименты

In [178]:
text = texts[0]

tokens_to_replace = get_text_synonyms(text, metric='cosine')
print('\nOriginal text:\n\n', text, '\n')
print('Modified text:\n\n', paste_synonyms(text, tokens_to_replace))

Original token             Closest token        value    is_distance_ok?    is_morph_ok?
-------------------------------------------------------------------------------------
проходит                   проходят             0.289616       x            -     
массовая                   всеобщая             0.449626       -            x     
манифестация               манифестации         0.414987       -            -     
в                          во                   0.365701       x            x     
поддержку                  поддержки            0.373150       x            -     
свободы                    свобода              0.360995       x            -     
передает                   передаёт             0.268194       x            -     
корреспондент              репортер             0.265512       x            x     
организованная             организована         0.318082       x            -     
при                        этом                 0.419719       -            - 

In [179]:
text = texts[1]

tokens_to_replace = get_text_synonyms(text, metric='cosine')
print('\nOriginal text:\n\n', text, '\n')
print('Modified text:\n\n', paste_synonyms(text, tokens_to_replace))

Original token             Closest token        value    is_distance_ok?    is_morph_ok?
-------------------------------------------------------------------------------------
эксперты                   специалисты          0.238146       x            x     
опрошенные                 допрошенные          0.439074       -            -     
разошлись                  разъехались          0.313827       x            -     
в                          во                   0.365701       x            x     
оценке                     оценки               0.333624       x            -     
актуальности               актуальность         0.335830       x            -     
для                        использовать         0.479390       -            -     
стратегических             ядерных              0.356429       x            x     
железнодорожных            поездов              0.354061       x            -     
ракетных                   ракет                0.299243       x            - 

In [180]:
text = texts[2]

tokens_to_replace = get_text_synonyms(text, metric='cosine')
print('\nOriginal text:\n\n', text, '\n')
print('Modified text:\n\n', paste_synonyms(text, tokens_to_replace))

Original token             Closest token        value    is_distance_ok?    is_morph_ok?
-------------------------------------------------------------------------------------
определится                определилась         0.417604       -            -     
с                          со                   0.278857       x            x     
местом                     убежищем             0.432312       -            x     
проведения                 проведении           0.250762       x            -     
финала                     финале               0.355929       x            -     
наций                      государств           0.356225       x            -     
года                       году                 0.193501       x            -     
декабря                    ноября               0.056676       x            x     
сообщается                 аль-бухари           0.432629       -            -     
на                         па                   0.498009       -            - 

## Выводы

Модель GloVe способна находить синонимы, но она очень плохо годится для этого. Например, в последнем тексте токены "декабрь" и "ноябрь" имеют очень малое расстояние между собой, что разумно для этих моделей, но не имеет смысла для синонимизации. Так как GloVe, как и W2V, основаны на том, как часто слова упоминаются в одном и том же контексте, и синонимы, и антонимы, и просто сходные по применению слова будут иметь похожие вектора.

Пример со словом "светлый", который имеет самый близкий токен "темный", а синонимы "солнечный" и "ясный" находятся не в начале списка:

In [200]:
ind = navec.vocab['светлый']

distances = pairwise_distances(emb_matrix, emb_matrix[ind:ind+1], metric='cosine')

sorted_indecies = distances[:,0].argsort()
sorted_values = distances[sorted_indecies,0]
for ind, value in zip(sorted_indecies[:10], sorted_values[:10]):
    print(f'{navec.vocab.words[ind]:>20}  {value:6f}')

             светлый  0.000000
              темный  0.279039
               серый  0.348976
              тёмный  0.373169
               синий  0.401431
               белый  0.432204
          золотистый  0.442950
           солнечный  0.449291
               ясный  0.462366
            красивый  0.463627
